In [1]:
import pickle
# from sympy import symbols, expand, lambdify, Expr, simplify, pprint, Rel
import sympy as sp
import math
from pyomo.environ import *
import pyomo.environ as pyo
from typing import List
import itertools
import numpy as np
from IPython.display import display

In [2]:
def affine_expr(coeffs, symbols):
    return sum(c * s for c, s in zip(coeffs[:-1], symbols)) + coeffs[-1]

In [3]:
def generate_region_combos(region_sizes, n_gl):
    """Generate region index combinations based on critical region structure."""
    n_theta = len(region_sizes)
    region_combo_shape = []
    for k in range(n_theta):
        n_paths = int(np.prod(n_gl[:k])) if k > 0 else 1
        region_combo_shape.extend([range(region_sizes[k])] * n_paths)
    return list(itertools.product(*region_combo_shape))

In [4]:
def compute_sf_expressions_region_combo_based(
        theta_bounds_list,
        theta_regions_list,
        joint_pdf_expr,
        d_syms,
        n_gl_list,
        theta_syms
):
    n_theta = len(theta_syms)
    quad_data = [np.polynomial.legendre.leggauss(n) for n in n_gl_list]
    region_sizes = [bounds.shape[0] for bounds in theta_bounds_list]
    region_combos = generate_region_combos(region_sizes, n_gl_list)

    sf_exprs = []
    sf_regions = []

    for region_combo in region_combos:
        combo_ptr = 0
        paths = [([], 1, 1, [])]  # (theta_vals, weight, scale, constraints)

        for level in range(n_theta):
            xi, wi = quad_data[level]
            new_paths = []

            # Determine how many paths we’re expanding at this level
            n_existing_paths = len(paths)
            for path_idx in range(n_existing_paths):
                theta_vals, weight, scale, constraints = paths[path_idx]
                region_idx = region_combo[combo_ptr]
                combo_ptr += 1

                # Compute bounds
                bound_inputs = theta_vals + list(d_syms)
                bounds = theta_bounds_list[level][region_idx]
                t_min = affine_expr(bounds[0], bound_inputs)
                t_max = affine_expr(bounds[1], bound_inputs)

                # Add region constraints
                rows = theta_regions_list[level][region_idx]
                new_constraints = list(constraints)
                for row in rows:
                    t_coeffs = row[:level]
                    d_coeffs = row[level:-1]
                    const = row[-1]
                    lhs = sum(c * theta_vals[i] for i, c in enumerate(t_coeffs)) + \
                          sum(c * d for c, d in zip(d_coeffs, d_syms)) + const
                    new_constraints.append(sp.simplify(lhs <= 0))

                for q in range(len(xi)):
                    t = 0.5 * (t_max - t_min) * xi[q] + 0.5 * (t_max + t_min)
                    new_theta_vals = theta_vals + [sp.simplify(t)]
                    new_weight = weight * wi[q]
                    new_scale = scale * 0.5 * (t_max - t_min)
                    new_paths.append((new_theta_vals, new_weight, new_scale, new_constraints))

            paths = new_paths

        # Final integration over all quadrature paths for this region combo
        sf_sum = 0
        for theta_vals, weight, scale, constraints in paths:
            theta_subs = {sym: val for sym, val in zip(theta_syms, theta_vals)}
            pdf_val = joint_pdf_expr.subs(theta_subs)
            sf_sum += weight * scale * pdf_val

        sf_exprs.append(sp.simplify(sf_sum))
        sf_regions.append(paths[0][3])  # constraints same for all in this combo

    return sf_exprs, sf_regions, len(region_combos)

In [5]:
# def compute_all_sf_expressions_generalized(
#         theta_bounds_list,  # List of shape (n_regions_i, 2, coeff_len) for each θᵢ
#         theta_regions_list,  # List of region constraint matrices for each θᵢ
#         joint_pdf_expr,  # sympy expression in θ₁...θₙ
#         d_syms,  # sympy symbols for uncertain parameters
#         n_gl_list,  # List of quadrature points per θᵢ
#         theta_syms  # [θ₁, θ₂, ..., θₙ]
# ):
#         
#         n_theta = len(theta_syms)
#         assert len(theta_bounds_list) == n_theta
#         assert len(theta_regions_list) == n_theta
#         assert len(n_gl_list) == n_theta
# 
#         quadrature_data = [np.polynomial.legendre.leggauss(n) for n in n_gl_list]
#         sf_exprs = []
#         sf_regions = []
# 
#         # First loop over θ₁ regions
#         n_t1_regions = theta_bounds_list[0].shape[0]
#         n_q1 = n_gl_list[0]
#         xi_1, wi_1 = quadrature_data[0]
# 
#         for t1_region_idx in range(n_t1_regions):
#             # θ₁ bounds (only depend on d_syms)
#             t1_min_expr = affine_expr(theta_bounds_list[0][t1_region_idx, 0], d_syms)
#             t1_max_expr = affine_expr(theta_bounds_list[0][t1_region_idx, 1], d_syms)
# 
#             # θ₁ quadrature points
#             # theta1_points = [
#             #     sp.simplify(0.5 * (t1_max_expr - t1_min_expr) * xi + 0.5 * (t1_max_expr + t1_min_expr))
#             #     for xi in xi_1
#             # ]
#             theta1_points = [
#                 0.5 * (t1_max_expr - t1_min_expr) * xi + 0.5 * (t1_max_expr + t1_min_expr) for xi in xi_1
#             ]
#             # θ₁ region constraints
#             t1_region_constraints = theta_regions_list[0][t1_region_idx]
#             t1_ineqs = [
#                 sum(c * d for c, d in zip(row[:-1], d_syms)) + row[-1] <= 0
#                 for row in t1_region_constraints
#             ]
# 
#             # For each θ₁ point, now we consider all combinations of θ₂ region indices (per θ₁ point)
#             n_t2_regions = theta_bounds_list[1].shape[0]
#             t2_region_choices = list(itertools.product(range(n_t2_regions), repeat=n_q1))
# 
#             for t2_combo in t2_region_choices:
#                 sf_sum = 0
#                 region_ineqs = list(t1_ineqs)
# 
#                 for i in range(n_q1):
#                     t1 = theta1_points[i]
#                     w1 = wi_1[i]
#                     t2_region_idx = t2_combo[i]
# 
#                     # θ₂ bounds depend on θ₁ and d
#                     t2_min_expr = affine_expr(theta_bounds_list[1][t2_region_idx, 0], [t1] + list(d_syms))
#                     t2_max_expr = affine_expr(theta_bounds_list[1][t2_region_idx, 1], [t1] + list(d_syms))
#                     xi_2, wi_2 = quadrature_data[1]
#                     n_q2 = n_gl_list[1]
# 
#                     # θ₂ region constraints
#                     for row in theta_regions_list[1][t2_region_idx]:
#                         θ1_coeff = row[0]
#                         d_coeffs = row[1:-1]
#                         const = row[-1]
#                         ineq = θ1_coeff * t1 + sum(c * d for c, d in zip(d_coeffs, d_syms)) + const <= 0
#                         # region_ineqs.append(sp.simplify(ineq))
#                         region_ineqs.append(ineq)
# 
#                     for j in range(n_q2):
#                         t2 = 0.5 * (t2_max_expr - t2_min_expr) * xi_2[j] + 0.5 * (t2_max_expr + t2_min_expr)
#                         # pdf_val = joint_pdf_expr.subs({theta_syms[0]: t1, theta_syms[1]: sp.simplify(t2)})
#                         pdf_val = joint_pdf_expr.subs({theta_syms[0]: t1, theta_syms[1]: t2})
#                         weight = w1 * wi_2[j]
#                         scale = 0.25 * (t1_max_expr - t1_min_expr) * (t2_max_expr - t2_min_expr)
#                         sf_sum += weight * scale * pdf_val
# 
#                 # sf_exprs.append(sp.simplify(sf_sum))
#                 sf_exprs.append(sf_sum)
#                 sf_regions.append(region_ineqs)
# 
#         return sf_exprs, sf_regions

In [6]:
def generate_constraints_from_expressions(
    expr_list: List[sp.Expr],
    region_list: List[List[sp.Expr]],
    instance: ConcreteModel,
    bounds_dict:dict,
    target: float = 1.0,
    big_m: float = 1e3
):
    """
    Adds constraint expressions to a Pyomo model using Big-M logic. For each expression in expr_list,
    the constraint is enforced only if the design variables lie in the corresponding critical region
    defined in region_list.

    Args:
        expr_list: List of SymPy expressions representing SF constraints.
        region_list: List of lists of SymPy inequality expressions defining valid regions.
        instance: Pyomo model (ConcreteModel).
        target: The minimum acceptable value for each SF expression.
        big_m: Big-M constant for constraint activation.
    """
    assert len(expr_list) == len(region_list), "Each expression must have a corresponding region definition."

    if not hasattr(instance, "generated_constraints"):
        instance.generated_constraints = ConstraintList()
    if not hasattr(instance, "region_constraints"):
        instance.region_constraints = ConstraintList()
    if not hasattr(instance, "region_binaries"):
        instance.region_binaries = Var(range(len(expr_list)), within=Binary)

    sf_expr_pyomo_list = list()

    for i, (sf_expr, region_exprs) in enumerate(zip(expr_list, region_list)):
        # Get all symbols in SF expression and region inequalities
        all_syms = sf_expr.free_symbols.union(*[reg.free_symbols for reg in region_exprs])
        all_syms = list(all_syms)

        # Ensure all symbols are added to the Pyomo model
        pyomo_vars = []
        for sym in all_syms:
            var_name = str(sym)
            if not hasattr(instance, var_name):
                setattr(instance, var_name, Var(bounds=bounds_dict[var_name]))
            pyomo_vars.append(getattr(instance, var_name))

        # Create a dict for substitution and lambdify
        sym_to_pyomo = {str(sym): getattr(instance, str(sym)) for sym in all_syms}
        lambdify_vars = list(sym_to_pyomo.keys())
        lambdify_vals = [sym_to_pyomo[s] for s in lambdify_vars]

        # Lambdify SF expression
        sf_func = sp.lambdify(lambdify_vars, sf_expr, modules=[{'exp': pyo.exp, 'pi': math.pi}, 'sympy'])
        sf_pyomo = sf_func(*lambdify_vals)
        sf_expr_pyomo_list.append(sf_pyomo)

        # Constraint: enforce SF ≥ target only when region binary = 1
        instance.generated_constraints.add(
            sf_pyomo >= target - big_m * (1 - instance.region_binaries[i])
        )

        # Region constraints: region_expr <= 0 + M*(1 - delta_i)
        for reg_expr in region_exprs:
            if not isinstance(reg_expr, sp.Rel):
                raise ValueError(f"Invalid region expression: {reg_expr} is not a relational (inequality) expression.")

            reg_func_lhs = sp.lambdify(lambdify_vars, reg_expr.lhs, modules='sympy')
            reg_func_rhs = sp.lambdify(lambdify_vars, reg_expr.rhs, modules='sympy')
            lhs_pyomo = reg_func_lhs(*lambdify_vals)
            rhs_pyomo = reg_func_rhs(*lambdify_vals)

            delta = instance.region_binaries[i]
            M_term = big_m * (1 - delta)

            if reg_expr.rel_op == '<=':
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo + M_term)
            elif reg_expr.rel_op == '<':
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo - 1e-6 + M_term)
            elif reg_expr.rel_op == '>=':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo - M_term)
            elif reg_expr.rel_op == '>':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo + 1e-6 - M_term)
            elif reg_expr.rel_op == '==':
                instance.region_constraints.add(lhs_pyomo >= rhs_pyomo - M_term)
                instance.region_constraints.add(lhs_pyomo <= rhs_pyomo + M_term)
            else:
                raise NotImplementedError(f"Unsupported relational operator: {reg_expr.rel_op}")

    # Optional: Only one region can be active
    instance.region_exclusivity = Constraint(expr=sum(instance.region_binaries[i] for i in range(len(expr_list))) == 1)
    if not hasattr(instance, 'sf'):
        instance.sf = Var(within=NonNegativeReals)

    if not hasattr(instance, 'sf_con'):
        instance.sf_con = Constraint(expr = instance.sf == sum(sf_expr_pyomo_list[i] * instance.region_binaries[i] for i in range(len(expr_list))))

In [7]:
# with open('factored_sf_exprs.pkl', 'rb') as file:
#     factored_sf_exprs = pickle.load(file)

In [8]:
# --- Define Symbols ---
theta_syms = sp.symbols('theta_1 theta_2')
d_syms = sp.symbols('d0 d1')
theta_1, theta_2 = theta_syms
d1, d2 = d_syms

n_gl_list = [3, 3]

# Bounds arrays
theta1_bounds_array = np.array([
    [[0.0, 0.0, 0.0], [0.0, 0.0, 4.0]],  # d0 ± 10
    [[0.75, -1.5, -1.25], [0.0, 0.0, 4.0]]  # d1 ± 5
])

theta2_bounds_array = np.array([
    [[-8/3, 2.0, -4.0, 2/3], [0.0, 0.0, 0.0, 4.0]],
    [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0]],
    [[1/3, -0.5, 0.5, -1/3], [0.0, 0.0, 0.0, 4.0]]
])

# PDF
joint_pdf_expr = (2/sp.pi) * sp.exp(-2 * ((theta_1 - 2) ** 2 + (theta_2 - 2) ** 2))

# Critical regions (CRs)
# theta1_critical_regions: constraints in terms of d (shape: n_regions × n_ineqs × (m+1))
theta1_critical_regions = np.array([
    [[1, -2, -5/3]],
    [[-1, 2, 5/3]]
], dtype=object)

# theta2_critical_regions: constraints in terms of [theta1, d0, d1, const]
theta2_critical_regions = np.array([
    [[-8/3, 2, -4, -10/3], [8/3, -2, 4, -2/3]],
    [[8/3, -4.0, 4.0, -8/3], [-8/3, 2, -4, 2/3]],
    [[-8/3, 4, -4, 8/3]]
], dtype=object)

# --- Setup lists ---
theta_bounds_list = [theta1_bounds_array, theta2_bounds_array]
theta_regions_list = [theta1_critical_regions, theta2_critical_regions]

In [9]:
sf_exprs, sf_regions, num_regions = compute_sf_expressions_region_combo_based(theta_bounds_list=theta_bounds_list, theta_regions_list=theta_regions_list,
                                                                              joint_pdf_expr=joint_pdf_expr, d_syms=[d1, d2], n_gl_list=n_gl_list,theta_syms=theta_syms)

In [10]:
print(f'Number of SF expressions: {len(sf_exprs)}')
print(f'Number of critical regions: {len(sf_regions)}')

Number of SF expressions: 54
Number of critical regions: 54


In [11]:
# for i, ex in enumerate(sf_exprs):
#     print(f'SF expression {i}')
#     display(ex)
#     print('\n')

In [12]:
# for i, region_exprs in enumerate(sf_regions):
#     print(f'For region {i}:')
#     for expr in region_exprs:
#         display(expr)
#     print('\n')
# # sf_regions

In [13]:
design_bounds = {
    'd0': (0,5),
    'd1':(0,5)
}

In [14]:
m = ConcreteModel()

In [15]:
generate_constraints_from_expressions(expr_list=sf_exprs, region_list=sf_regions, instance=m, bounds_dict=design_bounds, target=0.8)

In [16]:
m.obj = Objective(expr=-10*m.d0 + 10*m.d1, sense=minimize)

In [17]:
results = SolverFactory('gams', solver='baron').solve(m, tee=True)

--- Job model.gms Start 07/10/25 17:28:32 45.7.0 64fbf3ce WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\45\gmsprmNT.txt
--- GAMS Parameters defined
    Input C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpk89orbv1\model.gms
    Output C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpk89orbv1\output.lst
    ScrDir C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpk89orbv1\225a\
    SysDir C:\GAMS\45\
    CurDir C:\Users\SHIVAM~1.VED\AppData\Local\Temp\tmpk89orbv1\
    LogOption 3
Licensee: MUD - 30 User License                          G230830|0002AO-GEN
          Texas A&M University, Chemical Engineering                DC11194
          C:\GAMS\45\gamslice.txt
          License Admin: Jeff Polasek, j-polasek@tamu.edu                  
          The maintenance period of the license expired on Jun 25, 2024
          Please contact GAMS or your distributor for further information
Processor information: 1 socket(s), 16 core(s), and 24 thread(s) available
GAMS 45.7.0   Copyright (C) 1987-2024 

In [18]:
m.sf.pprint()

sf : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :     0 : 0.800000000002507 :  None : False : False : NonNegativeReals


In [19]:
m.d0.pprint()

d0 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   5.0 :     5 : False : False :  Reals


In [20]:
m.d1.pprint()

d1 : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :     0 : 0.926351864568996 :     5 : False : False :  Reals


In [21]:
m.region_binaries.pprint()

region_binaries : Size=54, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53}
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :   0.0 :     1 : False : False : Binary
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   0.0 :     1 : False : False : Binary
      6 :     0 :   0.0 :     1 : False : False : Binary
      7 :     0 :   0.0 :     1 : False : False : Binary
      8 :     0 :   0.0 :     1 : False : False : Binary
      9 :     0 :   0.0 :     1 : False : False : Binary
     10 :     0 :   0.0 :     1 : False : False : Binary
     11 :     0 :   0.0 :     1 : False : False : Binary
     12 :     0 :  

In [22]:
m.pprint()

4 Var Declarations
    d0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :     5 : False : False :  Reals
    d1 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 0.926351864568996 :     5 : False : False :  Reals
    region_binaries : Size=54, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False